# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [25]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.multiclass import type_of_target
import numpy as np
import pickle





nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eart_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eart_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eart_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_message', con=engine)
X = df['message'].values
Y = df.drop(['id','message','original','genre'],axis=1)
category_names = Y.columns
Y = Y.values

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
Y[500]

array([1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = text.strip().lower().translate(str.maketrans('', '', string.punctuation))

    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemmed

print(tokenize('Weather update - a cold front from Cuba that could pass over Haiti'))

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=100,n_jobs=-1))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024F1E410730>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
#type_of_target(y_test)
print(classification_report(y_test, y_pred,target_names=category_names))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      4000
               request       0.83      0.49      0.61       872
                 offer       0.00      0.00      0.00        21
           aid_related       0.79      0.54      0.64      2126
          medical_help       0.46      0.02      0.03       395
      medical_products       0.92      0.04      0.08       271
     search_and_rescue       0.25      0.01      0.01       131
              security       0.00      0.00      0.00        89
              military       0.71      0.03      0.06       173
                 water       0.90      0.27      0.41       328
                  food       0.88      0.44      0.59       560
               shelter       0.83      0.21      0.34       444
              clothing       0.78      0.09      0.16        78
                 money       0.75      0.03      0.05       111
        missing_people       1.00      

C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=50))
])

In [12]:
parameters = {
    'clf__n_estimators': [50, 100],
    'clf__max_depth': [5, 7]
}
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs= -1,scoring='accuracy', verbose=10)

In [13]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:  6.3min remaining:  9.5min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed: 14.0min remaining: 11.4min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed: 14.0min remaining:  6.0min
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed: 17.3min remaining:  3.1min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 17.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 17.4min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000024F1E410730>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        RandomForestClassifier(n_estimators=50))]),
             n_jobs=-1,
             param_grid={'clf__max_depth': [5, 7],
                         'clf__n_estimators': [50, 100]},
             scoring='accuracy', verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred = cv.predict(X_test)

In [15]:
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      4000
               request       0.00      0.00      0.00       872
                 offer       0.00      0.00      0.00        21
           aid_related       0.87      0.01      0.01      2126
          medical_help       0.00      0.00      0.00       395
      medical_products       0.00      0.00      0.00       271
     search_and_rescue       0.00      0.00      0.00       131
              security       0.00      0.00      0.00        89
              military       0.00      0.00      0.00       173
                 water       0.00      0.00      0.00       328
                  food       0.00      0.00      0.00       560
               shelter       0.00      0.00      0.00       444
              clothing       0.00      0.00      0.00        78
                 money       0.00      0.00      0.00       111
        missing_people       0.00      

C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
print(cv.best_estimator_)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024F1E410730>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(max_depth=5, n_estimators=50))])


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
parameters = parameters = {
    'tfidf__use_idf': [True, False],
    'clf__n_estimators': [50, 100]
}
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs= -1)


In [20]:

cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000024F1E410730>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        RandomForestClassifier(n_estimators=50))]),
             n_jobs=-1,
             param_grid={'clf__n_estimators': [50, 100],
                         'tfidf__use_idf': [True, False]})

In [21]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      4000
               request       0.85      0.50      0.63       872
                 offer       0.00      0.00      0.00        21
           aid_related       0.79      0.54      0.64      2126
          medical_help       0.29      0.01      0.02       395
      medical_products       0.89      0.03      0.06       271
     search_and_rescue       0.57      0.03      0.06       131
              security       0.00      0.00      0.00        89
              military       0.80      0.05      0.09       173
                 water       0.90      0.26      0.41       328
                  food       0.87      0.42      0.57       560
               shelter       0.85      0.23      0.36       444
              clothing       0.89      0.10      0.18        78
                 money       0.75      0.03      0.05       111
        missing_people       1.00      

C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
print(cv.best_estimator_)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024F1E410730>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])


### 9. Check other model

In [41]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100,max_depth=5)))
])
parameters = parameters = {
    'tfidf__use_idf': [True, False],
    'tfidf__sublinear_tf':[True, False]
}
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs= -1)



In [42]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000024F1E410730>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=5)))]),
             n_jobs=-1,
             param_grid={'tfidf__sublinear_tf': [True, False],
                         'tfidf__use_idf': [True, False]})

In [43]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      4000
               request       0.00      0.00      0.00       872
                 offer       0.00      0.00      0.00        21
           aid_related       0.92      0.01      0.01      2126
          medical_help       0.00      0.00      0.00       395
      medical_products       0.00      0.00      0.00       271
     search_and_rescue       0.00      0.00      0.00       131
              security       0.00      0.00      0.00        89
              military       0.00      0.00      0.00       173
                 water       0.00      0.00      0.00       328
                  food       0.00      0.00      0.00       560
               shelter       0.00      0.00      0.00       444
              clothing       0.00      0.00      0.00        78
                 money       0.00      0.00      0.00       111
        missing_people       0.00      

C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\eart_\.conda\envs\python3.6\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
print(cv.best_estimator_)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000024F1E410730>)),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=5)))])


### 10. Export your model as a pickle file

In [45]:
with open("../models/disaster_response_model.pkl", 'wb') as f:
    pickle.dump(pipeline, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.